# Implémentation GPU - Algorithme de hachage Fowler-Noll-Vo - 1A #
Sources : 
- http://www.isthe.com/chongo/tech/comp/fnv/index.html
- https://en.wikipedia.org/wiki/Fowler%E2%80%93Noll%E2%80%93Vo_hash_function
---

### Import des librairies externes ###

In [7]:
import numpy as np
from numba import cuda, uint64
import time

### Fonction FNV-1A - Kernel CUDA ###

In [8]:
@cuda.jit
def fnv1a_kernel(messages, lengths, output):
    """
    Implementation GPU de l'algorithme de hash FNV1a-64 bits
      Params : 
        messages : message a hasher
        lengths : taille du message
        output: hash de sortie
    """
    FNV1A_64_INIT = uint64(0xCBF29CE484222325) # decalage initial
    FNV_64_PRIME = uint64(0x100000001B3)       # Nb premier, pour 64bits

    # TID
    tid = cuda.threadIdx.x+ cuda.blockIdx.x * cuda.blockDim.x

    # Verification des dimensions
    if tid >= messages.shape[0]:
        return
        
    # init sur le decalage initial
    h = FNV1A_64_INIT
    for i in range(lengths[tid]): # hash le message en repercutant les bits
        h ^= uint64(messages[tid, i])  # XOR
        h *= FNV_64_PRIME  # Multiplication par le nb premier
        h &= uint64(0xFFFFFFFFFFFFFFFF)  # Assurer 64-bit

    # Message hashé
    output[tid] = h

### Fonction d'appel et de mesure du temps d'exécution ###

In [9]:
def hash_all_gpu(input_list):
    """
    Hash une liste de chaînes avec FNV-1a 64-bit sur GPU
      params:
       input_list: liste des messages en clair à hasher
    """
    print(f"Hash de {len(input_list):,} chaînes en GPU (FNV-1a 64-bit)...")

    N = len(input_list)
    max_len = 32  # chaînes de 32 caractères

    # Init les tableaux
    messages_np = np.zeros((N, max_len), dtype=np.uint8)
    lengths_np = np.zeros(N, dtype=np.uint8)

    # Remplis avec les messages à hasher
    for i, s in enumerate(input_list):
        encoded = s.encode('utf-8')[:max_len]
        messages_np[i, :len(encoded)] = list(encoded)
        lengths_np[i] = len(encoded)
    
    output_np = np.zeros(N, dtype=np.uint64)

    # alloc gpu
    d_messages = cuda.to_device(messages_np)
    d_lengths = cuda.to_device(lengths_np)
    d_output = cuda.device_array(N, dtype=np.uint64)

    Nthreads = 128
    Nblocks = int(np.ceil(N / Nthreads))

    # lancer le kernel
    start = time.time()
    fnv1a_kernel[Nblocks, Nthreads](d_messages, d_lengths, d_output)
    cuda.synchronize()
    end = time.time()

    # Resultats
    d_output.copy_to_host(ary=output_np)

    print(f"Temps total : {(end - start) * 1e3} ms")
    print(f"Exemple : {input_list[0]} -> {output_np[0]:016x}")
    return output_np


---
### Exécution standard ###

In [10]:
# remplir une liste de messages a hasher
if __name__ == "__main__":
    nb_messages = 1000000 # 1 million
    test_data = [f"data_{i}" for i in range(nb_messages)]  
    hashes_gpu = hash_all_gpu(test_data)

Hash de 1,000,000 chaînes en GPU (FNV-1a 64-bit)...


NvvmSupportError: libNVVM cannot be found. Do `conda install cudatoolkit`:
Could not find module 'nvvm.dll' (or one of its dependencies). Try using the full path with constructor syntax.